<h3>Парсинг объявлений о продаже автомобилей с Авито</h3>

In [8]:
# импорт библиотек
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException 

import time
from random import randint

import pandas as pd

In [9]:
# начальная страница парсинга
URL = 'https://www.avito.ru/samarskaya_oblast/avtomobili?f=ASgBAgECAUTyCrCKAQFF~owUF3siZnJvbSI6bnVsbCwidG8iOjIwMjB9'

Напишем функцию для решения Captcha

In [10]:
def kill_captcha(driver):
    wait = WebDriverWait(driver, 5)
    try:
        # Проверка на наличие капчи
        captcha_element = wait.until(EC.presence_of_element_located(
                    (By.XPATH, "//h2[@class='firewall-title']")), message="Капча обнаружена")

        if captcha_element:
            print("Капча обнаружена. Решите ее вручную.")
            input("Нажмите Enter, когда решите капчу...")

    except TimeoutException:
        pass

Напишем функцию получения данных с объявления 

In [11]:
def get_car_data(ad, mark):
    data = {}

    data['id'] = ad.get_attribute("id")
    data['mark'] = mark
    data['name'] = ad.find_element(
                        by=By.CSS_SELECTOR, value='h3[itemprop="name"]'
                    ).text

    data['price'] = ad.find_element(
                        by=By.CSS_SELECTOR, value='meta[itemprop="price"]'
                    ).get_attribute("content")

    data['params'] = ad.find_element(
                        by=By.CSS_SELECTOR, value='p[data-marker="item-specific-params"]'
                    ).text

    return  data

Напишем функцию парсинга для всех объявлений

In [12]:
def parse_cars(url):
    # Настраиваем параметры браузера
    options = webdriver.ChromeOptions()
    # Отключает функцию, указывающую сайтам, что браузер управляется автоматизацией
    options.add_argument("--disable-blink-features=AutomationControlled")
    driver = webdriver.Chrome(options=options)
    driver.implicitly_wait(10)

    driver.get(url)

    kill_captcha(driver)
    # парсим популярные модели
    popular_models = driver.find_element(
        by=By.CSS_SELECTOR, value='div[data-marker="popular-rubricator/links"]'
    ).find_elements(
        by=By.CSS_SELECTOR, value='a[data-marker="popular-rubricator/link"]'
    )

    models = [{'name': x.text, 'link': x.get_attribute('href')} for x in popular_models]
    
    all_cars = []
    for model in models:
        # перейдем к объявлениям марки
        driver.get(model['link'])

        kill_captcha(driver)

        cars = []
        while True:
            # делаем небольшую задержку
            rand_sleep = randint(25, 49)
            time.sleep(rand_sleep / 10)

            ads = driver.find_elements(
                by=By.CSS_SELECTOR, value='div[data-marker="item"]'
            )

            for ad in ads:
                car_data = get_car_data(ad, model['name'])
                cars.append(car_data)
    
            try:
                # Ожидаем, пока кнопка станет видимой и доступной
                button = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, 'a[data-marker="pagination-button/nextPage"]'))
                )

                if button.is_displayed() and button.is_enabled():
                    button.click()
                else:
                    break

            except Exception:
                break
                
        all_cars += cars
        pd.DataFrame(cars).to_csv(f"../data/brands/cars_data_{model['name']}.csv")
        
    driver.quit()
    return all_cars

*Авито* отображает максимум 100 страниц по 50 объявлений, поэтому мы проходимся по популярным брендам и парсим их по очереди, чтобы суммарно было больше объявлений.

In [13]:
# Парсинг
cars = parse_cars(URL)

Капча обнаружена. Решите ее вручную.


In [14]:
# сохраняем данные
cars = pd.DataFrame(cars)

cars.to_csv('../data/cars_data.csv')

In [15]:
cars.head()

,id,mark,name,price,params
0,i3675665511,Audi,"Audi Q5 2.0 AT, 2011, 227 020 км",1459000,"227 020 км, 2 AT (211 л.с.), внедорожник, полн..."
1,i4156063281,Audi,"Audi A5 2.0 AMT, 2020, 45 000 км",4600000,"45 000 км, 2 AMT (249 л.с.), купе, полный, бензин"
2,i3972915633,Audi,"Audi A4 1.8 CVT, 2013, 153 414 км",1350230,"153 414 км, 1.8 CVT (170 л.с.), седан, передни..."
3,i4152882630,Audi,"Audi A5 2.0 AMT, 2018, 73 400 км",3460000,"73 400 км, 2.0 AMT (190 л.с.), купе, полный, д..."
4,i4026986935,Audi,"Audi Q7 3.0 AT, 2020, 54 000 км",7650000,"54 000 км, 3.0 AT (249 л.с.), внедорожник, пол..."
